In [67]:
import pandas as pd

CAS75_re_lt = []

CAS75 = pd.DataFrame.from_csv("75_CAS.csv")
for i in CAS75.values.tolist() :
    Date_ID = i[2].replace("2015/","").replace("/","")
    CAS75_re_lt.append( i + [i[1] + "-" + Date_ID + "-" + str(i[3])])

df_re75 = pd.DataFrame(CAS75_re_lt)
df_re75 = df_re75.rename(columns={0: 'Class', 1: 'Sample',2:"Date",3: "Peak",4:"RT", 5: "scan_number",6 :"Intensity",\
                        7 : "Area%", 8 : "CAS_ID", 9 : "Quality",10 : "Peak_ID"})

# index number regenerating
print "Original CAS-ID number : ",len(df_re75.CAS_ID.unique())
print df_re75.shape,df_re75.size
df_re75.head()

Original CAS-ID number :  479
(8468, 11) 93148


,Class,Sample,Date,Peak,RT,scan_number,Intensity,Area%,CAS_ID,Quality,Peak_ID
0,CU,CU3,2015/11/03,6,8.203,2166,199396,0.26,000123-72-8,78.0,CU3-1103-6
1,CU,CU3,2015/11/03,8,8.380,2213,49279,0.07,000534-22-5,76.0,CU3-1103-8
2,CU,CU3,2015/11/03,9,8.533,2254,769124,1.02,000109-99-9,90.0,CU3-1103-9
3,CU,CU3,2015/11/03,10,8.593,2270,78929,0.10,000109-99-9,80.0,CU3-1103-10
4,CU,CU3,2015/11/03,12,8.897,2351,133877,0.18,000071-43-2,91.0,CU3-1103-12


In [68]:
ten_sec = 10*(1.0/60)
ten_sec2scan = ten_sec / (df_re75.RT/df_re75.scan_number)
ten_sec2scan_avg = ten_sec2scan.mean()

print "10 second to scan number : ", ten_sec2scan_avg

10 second to scan number :  44.2290571911


In [69]:
x = []
CAS_group = df_re75.groupby("CAS_ID").groups

print "CAS-Group type : ",type(CAS_group)


for cas in CAS_group :
    x.append([cas,CAS_group[cas]])
"~~~~~~~~~~~~~~~~~~show dataframe ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
test_group_df = pd.DataFrame(x)
test_group_df = test_group_df.rename(columns={0: 'CAS_ID', 1: 'Sample_index'})
test_group_df.head()

CAS-Group type :  <type 'dict'>


,CAS_ID,Sample_index
0,000105-68-0,"[40, 110, 169, 214, 279, 339, 397, 462, 529, 6..."
1,000100-52-7,"[395, 706, 776, 926, 1300, 1607, 1783, 1836, 2..."
2,000142-92-7,"[46, 113, 173, 221, 282, 343, 400, 467, 540, 6..."
3,000111-27-3,"[329, 384, 1431, 1771, 1829, 2255, 2317, 2438,..."
4,006117-97-1,[3090]


In [70]:
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth


def MeanScan(CAS_ID, cas_index_list , scan_list , bandwidth ) :
    # Y has index number of data frame
    Y = np.array(zip(cas_index_list,np.zeros(len(cas_index_list))), dtype=np.int)    
    X = np.array(zip(scan_list,np.zeros(len(scan_list))), dtype=np.int)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(X)
    # label of item
    labels = ms.labels_
    cluster_centers = ms.cluster_centers_
    labels_unique = np.unique(labels)
    n_clusters_ = len(labels_unique)
    # Let be check
    result = []
    for k in range(n_clusters_):
        my_members = labels == k
        result.append( [CAS_ID , k , list(Y[my_members, 0]) , list(X[my_members, 0])] )
        # CAS-ID  / sub cluster / dataframe index / real scan value
    return result

In [71]:
pre_group = []
bandwidth = ten_sec2scan_avg*3 # *8

for sub in CAS_group :
    CAS_ID = sub
    cas_index_list = CAS_group[sub] 
    scanValue_list = list( df_re75.ix[ cas_index_list ].scan_number.values )
    L = MeanScan( CAS_ID ,cas_index_list ,scanValue_list ,bandwidth) 
    pre_group = pre_group + L

# Put all list to pre_group

In [97]:
GroupedCAS = []

for sgro in pre_group :
    # CAS-ID  / sub cluster / dataframe index / count() / scan number mean / Quality mean / Quality max
    casID = sgro[0]
    subcl_num = sgro[1]
    indeData = sgro[2]
    ScanAVG = np.mean(sgro[3])
    G_sub = df_re75.ix[indeData].Quality
    GroupedCAS.append( [casID , subcl_num , indeData , len(indeData) , ScanAVG , G_sub.mean() , G_sub.max()] )

"~~~~~~~~~~~~~~~~~~show dataframe ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
GroupedCAS_Df = pd.DataFrame(GroupedCAS)
GroupedCAS_Df = GroupedCAS_Df.rename(columns={0: 'CAS_ID', 1: 'Sub_Cluster' ,2: "Sample_index" , 3: "Count",\
                                             4: "scanValue_avg" , 5: "QualMean", 6: "QualMax"})
# GroupedCAS_Df

print "Original Size : " ,GroupedCAS_Df.shape,GroupedCAS_Df.size
# df[ GroupedCAS_Df['QualMax']>= 90]
tet = GroupedCAS_Df[ GroupedCAS_Df['QualMax']>= 90]
V = tet[tet["Count"]>=3]

print "More than 3 & MaxQuality over 90 : " , V.shape ,"합칠거 고려시 감소 예정"
print "More than 3 & MaxQuality over 90 Unique CAS ID number: " , len(V.CAS_ID.unique())
print "Original sample : ",df_re75.shape ," /Used sample : ", V.Count.sum()

V.sort_index(by=['QualMax',"Count"], ascending=[False,False]).head()
# V.head()
# GroupedCAS_Df.head()

Original Size :  (861, 7) 6027
More than 3 & MaxQuality over 90 :  (351, 7) 합칠거 고려시 감소 예정
More than 3 & MaxQuality over 90 Unique CAS ID number:  181
Original sample :  (8468, 11)  /Used sample :  7161


/home/madbunny/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:26: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


,CAS_ID,Sub_Cluster,Sample_index,Count,scanValue_avg,QualMean,QualMax
182,000644-30-4,0,"[64, 239, 562, 622, 1069, 1324, 1402, 1700, 22...",45,9048.177778,96.244444,99.0
122,001430-97-3,0,"[74, 77, 80, 243, 245, 250, 566, 627, 630, 632...",159,10728.534591,94.132075,98.0
663,001730-37-6,0,"[72, 75, 78, 244, 246, 249, 251, 568, 629, 631...",149,10723.536913,94.255034,98.0
664,001730-37-6,1,"[131, 134, 303, 306, 308, 311, 361, 426, 429, ...",103,9934.572816,93.601942,98.0
123,001430-97-3,1,"[130, 133, 135, 302, 305, 307, 310, 425, 428, ...",95,9939.494737,93.105263,98.0


# How to .... 이성질체 문제

## 높은 퀄리티의 많은 갯수의 물질들
http://webbook.nist.gov/cgi/cbook.cgi?ID=001730-37-6&Units=SI
    
http://webbook.nist.gov/cgi/cbook.cgi?ID=001430-97-3&Units=SI

In [99]:
import numpy as np
from sklearn.cluster import MeanShift

iso_scan = V.scanValue_avg.tolist()
x = iso_scan
X = np.array(zip(x,np.zeros(len(x))), dtype=np.float)
y = [ i for i in range(len(iso_scan)) ] 
Y = np.array(zip(y,np.zeros(len(y))), dtype=np.int)

bandwidth = 10
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

Z = []

for k in range(n_clusters_):
    my_members = labels == k
    Z.append([ list(Y[my_members, 0]),len(list(Y[my_members, 0]))  ,list( X[my_members, 0]  ) ])

    

"~~~~~~~~~~~~~~~~~~show dataframe ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
test_iso_df = pd.DataFrame(Z)
test_iso_df = test_iso_df.rename(columns={0: 'Cluster_Table_index',1: "Count" ,2: 'scan_number'})

print "Things to Check : ",len(test_iso_df.Cluster_Table_index) , " We would get 170 compound"
test_iso_df.head()




Things to Check :  170  We would get 170 compound


,Cluster_Table_index,Count,scan_number
0,"[30, 65, 183, 204, 255, 280, 282]",7,"[2736.0, 2731.5, 2740.5, 2744.77777778, 2740.7..."
1,"[108, 213, 234, 283, 320, 324, 346]",7,"[2329.05882353, 2326.4, 2325.83018868, 2313.25..."
2,"[0, 81, 221, 237, 240, 242, 286, 292, 306]",9,"[4383.5, 4376.1, 4376.0, 4369.6, 4385.45454545..."
3,"[20, 132, 159, 239, 287]",5,"[5206.61363636, 5208.30952381, 5205.62162162, ..."
4,"[11, 94, 202, 215, 224]",5,"[12011.0, 12022.5555556, 12016.3333333, 12019...."


# Manual 하게 하는게 짜는것 보다 빠르겠다

In [56]:
df_re75.head()

,Class,Sample,Date,Peak,RT,scan_number,Intensity,Area%,CAS_ID,Quality,Peak_ID
0,CU,CU3,2015/11/03,6,8.203,2166,199396,0.26,000123-72-8,78.0,CU3-1103-6
1,CU,CU3,2015/11/03,8,8.380,2213,49279,0.07,000534-22-5,76.0,CU3-1103-8
2,CU,CU3,2015/11/03,9,8.533,2254,769124,1.02,000109-99-9,90.0,CU3-1103-9
3,CU,CU3,2015/11/03,10,8.593,2270,78929,0.10,000109-99-9,80.0,CU3-1103-10
4,CU,CU3,2015/11/03,12,8.897,2351,133877,0.18,000071-43-2,91.0,CU3-1103-12



### 왜 몇몇은 같은 걸텐데 피크 타임이 몇 분 / 몇 초씩 차이나는 걸까?

피크 순서로 봤을때는 아마도 같은 물질인 것이다. 하지만 날자에 따라서 다르게 나타나는 경향을 보인다.


In [15]:
A1 = GroupedCAS_Df.ix[0].Sample_index
A2 = GroupedCAS_Df.ix[1].Sample_index
A3 = GroupedCAS_Df.ix[2].Sample_index

In [22]:
df_re75.ix[A1].sort_index(by=['Date'], ascending=[False])

/home/madbunny/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


,Class,Sample,Date,Peak,RT,scan_number,Intensity,Area%,CAS_ID,Quality,Peak_ID
2522,MU,MU2,2015/11/11,42,16.435,4362,577952,0.50,000105-68-0,86.0,MU2-1111-42
1147,CU,CU1,2015/11/11,47,16.432,4361,343351,0.79,000105-68-0,80.0,CU1-1111-47
4615,MR,MR4,2015/11/10,43,16.420,4358,2751838,3.33,000105-68-0,90.0,MR4-1110-43
6002,CR,CR2,2015/11/10,39,16.443,4364,658760,0.79,000105-68-0,80.0,CR2-1110-39
5735,CR,CR4,2015/11/10,45,16.424,4359,1695813,1.19,000105-68-0,86.0,CR4-1110-45
5536,CR,CR3,2015/11/10,45,16.424,4359,2048288,2.06,000105-68-0,86.0,CR3-1110-45
5078,CR,CR5,2015/11/10,41,16.420,4358,1616338,1.44,000105-68-0,86.0,CR5-1110-41
3960,MR,MR5,2015/11/10,50,16.420,4358,1476497,0.72,000105-68-0,90.0,MR5-1110-50
3212,MR,MR2,2015/11/10,48,16.420,4358,1408009,0.81,000105-68-0,90.0,MR2-1110-48
7651,CD,CD7,2015/11/09,42,16.469,4371,429518,0.60,000105-68-0,86.0,CD7-1109-42


In [23]:
df_re75.ix[A2].sort_index(by=['Date'], ascending=[False])

/home/madbunny/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


,Class,Sample,Date,Peak,RT,scan_number,Intensity,Area%,CAS_ID,Quality,Peak_ID
339,CU,CU4,2015/11/26,42,14.767,3917,214483,0.43,000105-68-0,83.0,CU4-1126-42
4688,MR,MR4,2015/11/26,42,14.752,3913,391935,1.83,000105-68-0,90.0,MR4-1126-42
4226,MR,MR1,2015/11/26,40,14.778,3920,69851,0.83,000105-68-0,83.0,MR1-1126-40
1102,CU,CU5,2015/11/26,36,14.763,3916,205247,1.59,000105-68-0,83.0,CU5-1126-36
4039,MR,MR5,2015/11/26,43,14.767,3917,170851,0.22,000105-68-0,86.0,MR5-1126-43
1609,MU,MU3,2015/11/26,42,14.767,3917,102556,0.52,000105-68-0,90.0,MU3-1126-42
3354,MR,MR2,2015/11/26,43,14.767,3917,224659,0.24,000105-68-0,80.0,MR2-1126-43
2268,MU,MU2,2015/11/26,43,14.763,3916,196664,0.34,000105-68-0,90.0,MU2-1126-43
110,CU,CU3,2015/11/25,35,14.767,3917,585024,2.14,000105-68-0,86.0,CU3-1125-35
1558,MU,MU1,2015/11/25,41,14.790,3923,150124,0.29,000105-68-0,83.0,MU1-1125-41


In [24]:
df_re75.ix[A3].sort_index(by=['Date'], ascending=[False])

/home/madbunny/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


,Class,Sample,Date,Peak,RT,scan_number,Intensity,Area%,CAS_ID,Quality,Peak_ID
169,CU,CU3,2015/12/17,43,17.560,4662,198689,0.28,000105-68-0,86.0,CU3-1217-43
462,CU,CU4,2015/12/17,46,17.568,4664,78988,0.17,000105-68-0,86.0,CU4-1217-46
980,CU,CU5,2015/12/17,45,17.567,4664,79064,0.53,000105-68-0,90.0,CU5-1217-45
4559,MR,MR4,2015/12/14,45,17.594,4671,173036,0.65,000105-68-0,90.0,MR4-1214-45
4871,CR,CR5,2015/12/14,44,17.590,4670,287446,0.98,000105-68-0,86.0,CR5-1214-44
5146,CR,CR1,2015/12/14,45,17.594,4671,341737,0.92,000105-68-0,86.0,CR1-1214-45
5601,CR,CR3,2015/12/14,42,17.601,4673,94554,0.48,000105-68-0,90.0,CR3-1214-42
5668,CR,CR4,2015/12/14,49,17.590,4670,434782,0.38,000105-68-0,86.0,CR4-1214-49
6086,CR,CR2,2015/12/14,46,17.594,4671,192735,0.17,000105-68-0,80.0,CR2-1214-46
3820,MR,MR5,2015/12/11,48,17.624,4679,199729,0.14,000105-68-0,90.0,MR5-1211-48


# Think about

다시 한번 더 돌려서 CAS-ID 와 QualMax 가 같고, scan value 평균이 비슷한 걸 하나의 클러스터로 묶어 보자 그게 합리적일 꺼 같아.

### 표준편차 적용할까?

그리고, 그 뒤에 피크 아이디 가 같다면, 제거해 나가는 식으로 진행 해보자

# 해야할 일 

### 1. 서로 다른 클러스터 중 같지 않은 것 제거
### 2. 중복된 Peak ID 를 가지는 피크 제거
### 3. 비슷한 시간대에 용출되는 물질 상관관계

# Peak Removal

## To Do

### 1. 사용할 sample group 과 버릴 sample group 나누기
### 2. 사용할 sample group 의 경우,  Peak-ID를 공유하는 left over sample 제거

사용할 피크일 경우에는  peak quality 를 group max 로 설정

합칠 클러스터는 메뉴얼 하게 데이터 보면서 해야 되나 아아아 ㅠㅠ


현제 하나에 피크에는 예상 가능한 서로 다른 후보 CAS-ID 가 속해있다.

In [30]:
y = []
PK_group = df_re75.groupby("Peak_ID").groups


for pkID in PK_group :
    y.append([pkID,PK_group[pkID]])
"~~~~~~~~~~~~~~~~~~show dataframe ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
test_pk_df = pd.DataFrame(y)
test_pk_df = test_pk_df.rename(columns={0: 'Peak_ID', 1: 'Sample_index'})


print "Size : ",test_pk_df.size
test_pk_df.head()

Size :  12514


,Peak_ID,Sample_index
0,CR3-1110-97,"[5574, 5575, 5576]"
1,CR3-1110-96,"[5572, 5573]"
2,CR3-1110-95,"[5570, 5571]"
3,CR3-1110-94,"[5567, 5568, 5569]"
4,CR3-1110-91,[5566]


In [25]:
df_re75.head()

,Class,Sample,Date,Peak,RT,scan_number,Intensity,Area%,CAS_ID,Quality,Peak_ID
0,CU,CU3,2015/11/03,6,8.203,2166,199396,0.26,000123-72-8,78.0,CU3-1103-6
1,CU,CU3,2015/11/03,8,8.380,2213,49279,0.07,000534-22-5,76.0,CU3-1103-8
2,CU,CU3,2015/11/03,9,8.533,2254,769124,1.02,000109-99-9,90.0,CU3-1103-9
3,CU,CU3,2015/11/03,10,8.593,2270,78929,0.10,000109-99-9,80.0,CU3-1103-10
4,CU,CU3,2015/11/03,12,8.897,2351,133877,0.18,000071-43-2,91.0,CU3-1103-12
